# Demo Tests OPX CW EDSR, Rabi etc

## First Tests with OPX+ :)

In [22]:
from qm.qua import *
from opx_driver import OPX
from configuration import *
from qualang_tools.addons.variables import assign_variables_to_element
from qualang_tools.loops import from_array
#####################################
#           Qcodes set-up           #
#####################################

# Create the OPX instrument class
opx_instrument = OPX(config, name="OPX_demo", host=qop_ip)
# Add the OPX instrument to the qcodes station
station.add_component(opx_instrument)



2023-04-18 13:06:26,507 - qm - INFO - Performing health check
2023-04-18 13:06:26,520 - qm - INFO - Health check passed
Connected to: Quantum Machines OPX+ in 0.67s. QOP Version = 2.40-8521884, SDK Version = 1.0.1.


'OPX_demo'

In [23]:
#####################################
#    EDSR - R&S Vector Source       #
#####################################
# Demonstrate how to perform a single point measurement with the OPX
# by integrating the readout signals while performing an external 2D sweep
# with the do2d function.




def OPX_pulsed_EDSR(Campl_in = 0.3 , n_avg_in = 10 , IV_buffer_len_in = 15_000_000, simulate = True):
    IV_buffer_len = IV_buffer_len_in # in ns to give IV converter enough time to stabilize 
    Campl = Campl_in
    n_avg = n_avg_in  # 10 is equivalent to approx 400 ms pulses -> 100 ms measurement time 
    N = int((IV_buffer_len + readout_len+4_000)/(2*bias_length)) # number of pulse cycles
    with program() as prog:
        I = declare(fixed)
        n = declare(int)
        I_on = declare(fixed)
        I_off = declare(fixed)
        I_out = declare(fixed)
        I_st = declare_stream()
        flag = declare(bool)
        counter = declare(int)
        assign_variables_to_element("readout_element", I)
        with infinite_loop_():
            if not simulate:
                    pause()
            with for_(n , 0 , n < n_avg , n + 1):  # averaging sequences
                assign(I_on, 0)
                assign(I_off, 0)
                with for_(counter, 0, counter < N, counter + 1):   # playing coulomb & IQ-pulse
                    play("bias"*amp(1*Campl), "gate_1")
                    #wait(5 // 4, "qubit")  # time unit in clock cycles (4ns)
                    #with if_(flag):
                    play("cw", "qubit")
                    play("bias"*amp(-1*Campl), "gate_1")
                    #wait((200-22) // 4 , "gate_1")
                wait(IV_buffer_len//4, "readout_element")
                measure(
                    "readout",
                    "readout_element",
                    None,
                    integration.full("cos", I, "out1"),
                )
                assign(I_on, I)
                with for_(counter, 0, counter < N, counter + 1):   # playing only coulomb-pulse
                    play("bias"*amp(1*Campl), "gate_1")
                    #wait(5 // 4, "qubit")  # time unit in clock cycles (4ns)
                    play("bias"*amp(-1*Campl), "gate_1")
                wait(IV_buffer_len//4, "readout_element")
                measure(
                    "readout",
                    "readout_element",
                    None,
                    integration.full("cos", I, "out1"),
                )
                assign(I_off, I)
                assign(I_out, I_on-I_off)
                save(I_out, I_st)

        with stream_processing():
            I_st.buffer(n_avg).map(FUNCTIONS.average()).save_all("I")
    return prog

In [46]:
#####################################
#    Rabi - R&S Vector Source       #
#####################################
# Demonstrate how to perform a single point measurement with the OPX
# by integrating the readout signals while performing an external 2D sweep
# with the do2d function.




def OPX_Rabi(t_burst_0 , t_burst_end , n_steps , Campl_in = 0.3 , n_avg_in = 10 , IV_buffer_len_in = 15_000_000, simulate = True):
    IV_buffer_len = IV_buffer_len_in # in ns to give IV converter enough time to stabilize 
    Campl = Campl_in
    n_avg = n_avg_in  # 10 is equivalent to approx 400 ms pulses -> 100 ms measurement time 
    N = int((IV_buffer_len + readout_len+4_000)/(2*bias_length)) # number of pulse cycles
    t_bursts = np.arange(t_burst_0,t_burst_end+(-t_burst_0+t_burst_end)//n_steps,(-t_burst_0+t_burst_end)//n_steps)
    with program() as prog:
        I = declare(fixed)
        n = declare(int)
        t = declare(int)
        I_on = declare(fixed)
        I_off = declare(fixed)
        I_out = declare(fixed)
        I_st = declare_stream()
        flag = declare(bool)
        counter = declare(int)
        assign_variables_to_element("readout_element", I)
        with infinite_loop_():
            if not simulate:
                    pause()
            with for_each_(t,t_bursts):
                with for_(n , 0 , n < n_avg , n + 1):  # averaging sequences
                    assign(I_on, 0)
                    assign(I_off, 0)
                    with for_(counter, 0, counter < N, counter + 1):   # playing coulomb & IQ-pulse
                        play("bias"*amp(1*Campl), "gate_1")
                        #wait(5 // 4, "qubit")  # time unit in clock cycles (4ns)
                        #with if_(flag):
                        play("cw", "qubit", duration = t)
                        play("bias"*amp(-1*Campl), "gate_1")
                        #wait((200-22) // 4 , "gate_1")
                    wait(IV_buffer_len//4, "readout_element")
                    measure(
                        "readout",
                        "readout_element",
                        None,
                        integration.full("cos", I, "out1"),
                    )
                    assign(I_on, I)
                    with for_(counter, 0, counter < N, counter + 1):   # playing only coulomb-pulse
                        play("bias"*amp(1*Campl), "gate_1")
                        #wait(5 // 4, "qubit")  # time unit in clock cycles (4ns)
                        play("bias"*amp(-1*Campl), "gate_1")
                    wait(IV_buffer_len//4, "readout_element")
                    measure(
                        "readout",
                        "readout_element",
                        None,
                        integration.full("cos", I, "out1"),
                    )
                    assign(I_off, I)
                    assign(I_out, I_on-I_off)
                    save(I_out, I_st)

        with stream_processing():
            I_st.buffer(n_avg).map(FUNCTIONS.average()).save_all("I")
    return prog

In [47]:
opx_instrument.qua_program = OPX_Rabi(t_burst_0 = 4, t_burst_end = 72, n_steps = 8, simulate=False, Campl_in = 0.5)

In [48]:
do1d(
    VS_freq, 2e9, 3e9, 20, 1,
    ISD,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    do_plot=True,
    exp=experiment,
)

2023-04-18 13:54:43,352 - qm - INFO - Flags: 
2023-04-18 13:54:43,353 - qm - INFO - Sending program to QOP
2023-04-18 13:55:31,236 - qm - INFO - Executing program
Starting experimental run with id: 645. Using 'qcodes.dataset.do1d'


  0%|          | 0/20 [00:00<?, ?it/s]

An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-0187943851aa;
Traceback:
Traceback (most recent call last):
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\dataset\dond\do_1d.py", line 162, in do1d
    (param_set, set_point), *call_param_meas(), *additional_setpoints_data
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\dataset\threading.py", line 109, in _call_params
    output.append((parameter, parameter.get()))
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\parameters\parameter_base.py", line 583, in get_wrapper
    raise e
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\parameters\parameter_base.py", line 570, in get_wrapper
    raw_value = get_function(*args, **kwargs)
  File "C:\Users\ZumAdmin\Documents\GitHub\SPIN_qcodes_stations\phys-dots-40\Rafael\T7Dc6\opx_driver.py", line 529, in get_raw
    result = self._instr.get_res()
  File "C:\Users\ZumAdmin\Docum

TypeError: ("unsupported operand type(s) for /: 'float' and 'NoneType'", 'getting OPX_results')

In [24]:
def OPX_pulsed_EDSR_Continuous(Campl_in = 0.3 , n_avg_in = 10 , IV_buffer_len_in = 15_000_000, simulate = True):
    
    IV_buffer_len = IV_buffer_len_in # in ns to give IV converter enough time to stabilize 
    Campl = Campl_in
    n_avg = n_avg_in  # 10 is equivalent to approx 400 ms pulses -> 100 ms measurement time 
    N = int((IV_buffer_len + readout_len+4_000)/(2*bias_length)) # number of pulse cycles
   
    with program() as prog:
        counter = declare(int)
        with infinite_loop_():
                with for_(counter, 0, counter < N, counter + 1):   # playing coulomb & IQ-pulse
                    play("bias"*amp(1*Campl), "gate_1")
                    play("cw", "qubit")
                    play("bias"*amp(-1*Campl), "gate_1")
                    
                
                with for_(counter, 0, counter < N, counter + 1):   # playing only coulomb-pulse
                    play("bias"*amp(1*Campl), "gate_1")
                    play("bias"*amp(-1*Campl), "gate_1")
            
    return prog

In [23]:
def OPX_pulsed_EDSR_Continuous_Lockin(Campl_in = 0.3 , n_avg_in = 10 , IV_buffer_len_in = 15_000_000, simulate = True):
    
    IV_buffer_len = IV_buffer_len_in # in ns to give IV converter enough time to stabilize 
    Campl = Campl_in
    n_avg = n_avg_in  # 10 is equivalent to approx 400 ms pulses -> 100 ms measurement time 
    N = int((IV_buffer_len + readout_len+4_000)/(2*bias_length)) # number of pulse cycles
   
    with program() as prog:
        counter = declare(int)
        with infinite_loop_():  # playing coulomb & IQ-pulse
                play("bias"*amp(1*Campl), "gate_1")
                play("cw", "qubit")
                play("bias"*amp(-1*Campl), "gate_1")       
    return prog

In [39]:
opx_instrument.qua_program = OPX_pulsed_EDSR_Continuous_Lockin(simulate=False, Campl_in = 0.5)

In [40]:
do1d(
    VS_freq, 2e9, 3e9, 20, 1,
    ISD,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    #do_plot=True,
    exp=experiment,
)

2023-04-18 13:49:47,060 - qm - INFO - Flags: 
2023-04-18 13:49:47,062 - qm - INFO - Sending program to QOP
2023-04-18 13:49:47,090 - qm - INFO - Executing program
Starting experimental run with id: 642. Using 'qcodes.dataset.do1d'


  0%|          | 0/20 [00:00<?, ?it/s]

An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-018794331102;
Traceback:
Traceback (most recent call last):
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\dataset\dond\do_1d.py", line 160, in do1d
    time.sleep(delay)
KeyboardInterrupt

Measurement has been interrupted, data may be incomplete: 


(results #642@C:\Users\ZumAdmin\Documents\GitHub\SPIN_qcodes_stations\phys-dots-40\Rafael\T7Dc6\T7D_c6_3gate_Finfet.db
 ---------------------------------------------------------------------------------------------------------------------
 VS_VS_freq - numeric
 ISD - numeric,
 (None,),
 (None,))

In [58]:

def simulate_OPX(program):
    opx_instrument.readout_pulse_length(readout_len)
    # Add the custom sequence to the OPX
    opx_instrument.qua_program = program(simulate = True)
    # Simulate program
    opx_instrument.sim_time(10_000)
    opx_instrument.simulate()
    opx_instrument.plot_simulated_wf()
    


In [ ]:
simulate_OPX(OPX_pulsed_EDSR_readout)

In [24]:
VS_pwr(-20)

In [38]:
# Execute program
opx_instrument.qua_program = OPX_0d_scan(simulate=False)
do1d(
    VS_freq, 2e9, 3e9, 100, 1,
    ISD,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    #do_plot=True,
    exp=experiment,
)

NameError: name 'OPX_0d_scan' is not defined

In [48]:
opx_instrument.qua_program = OPX_0d_scan(simulate=False)

VB(-1.205)
VS_pwr(-80)
do2d(
    VP1,-2.3,-2.7,100,1,
    VP2,-1.05,-1.35,100,0.041333,
    ISD,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    #do_plot=True,
    exp=experiment,
)

2023-04-05 22:16:05,201 - qm - INFO - Flags: 
2023-04-05 22:16:05,202 - qm - INFO - Sending program to QOP
2023-04-05 22:16:51,252 - qm - INFO - Executing program
Starting experimental run with id: 529. Using 'qcodes.dataset.do2d'


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

(results #529@C:\Users\ZumAdmin\Documents\GitHub\SPIN_qcodes_stations\phys-dots-40\Rafael\T7Dc6\T7D_c6_3gate_Finfet.db
 ---------------------------------------------------------------------------------------------------------------------
 LNHR_dac_chan6_V_P1 - numeric
 LNHR_dac_chan7_V_P2 - numeric
 ISD - numeric
 I - numeric,
 (None,),
 (None,))

In [49]:
opx_instrument.qua_program = OPX_0d_scan(simulate=False)

VB(-1.205)
VS_pwr(-10)
do2d(
    VP1,-2.3,-2.7,100,1,
    VP2,-1.05,-1.35,100,0.041333,
    ISD,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    #do_plot=True,
    exp=experiment,
)

2023-04-06 04:09:59,262 - qm - INFO - Flags: 
2023-04-06 04:09:59,264 - qm - INFO - Sending program to QOP
2023-04-06 04:10:45,213 - qm - INFO - Executing program
Starting experimental run with id: 530. Using 'qcodes.dataset.do2d'


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

(results #530@C:\Users\ZumAdmin\Documents\GitHub\SPIN_qcodes_stations\phys-dots-40\Rafael\T7Dc6\T7D_c6_3gate_Finfet.db
 ---------------------------------------------------------------------------------------------------------------------
 LNHR_dac_chan6_V_P1 - numeric
 LNHR_dac_chan7_V_P2 - numeric
 ISD - numeric
 I - numeric,
 (None,),
 (None,))

In [66]:
Campls = [0,0.1,0.2,0.3,0.4,0.5]
for i in Campls:
    opx_instrument.qua_program = OPX_pulsed_EDSR_Continuous(simulate=False, Campl_in = i)
    VB(-1.205)
    VS_pwr(-20)
    do2d(
        VP1,-2.3,-2.7,100,1,
        VP2,-1.,-1.5,100,0.041333,
        ISD,
        enter_actions=[opx_instrument.run_exp],
        exit_actions=[opx_instrument.halt],
        show_progress=True,
        exp=experiment,
    )

2023-04-06 12:03:41,884 - qm - INFO - Flags: 
2023-04-06 12:03:41,886 - qm - INFO - Sending program to QOP
2023-04-06 12:03:41,918 - qm - INFO - Executing program
Starting experimental run with id: 535. Using 'qcodes.dataset.do2d'


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

An exception occured in measurement with guid: aaaaaaaa-0000-0000-0000-018756565e40;
Traceback:
Traceback (most recent call last):
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\dataset\dond\do_2d.py", line 200, in do2d
    *call_param_meas(),
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\dataset\threading.py", line 109, in _call_params
    output.append((parameter, parameter.get()))
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\parameters\parameter_base.py", line 570, in get_wrapper
    raw_value = get_function(*args, **kwargs)
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\parameters\scaled_paramter.py", line 171, in get_raw
    wrapped_value = cast(float, self._wrapped_parameter())
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\site-packages\qcodes\parameters\parameter_base.py", line 368, in __call__
    return self.get()
  File "C:\Users\ZumAdmin\anaconda3\envs\qcodes\lib\

In [68]:
opx_instrument.qua_program = OPX_pulsed_EDSR(Campl_in = 0.2, simulate = False)

VB(-1.205)
VS_pwr(-10)
do2d(
    VP1,-2.3,-2.7,50,1,
    VP2,-1.05,-1.35,50,0.041333,
    ISD,
    opx_instrument.resume,
    opx_instrument.get_measurement_parameter(),
    enter_actions=[opx_instrument.run_exp],
    exit_actions=[opx_instrument.halt],
    show_progress=True,
    #do_plot=True,
    exp=experiment,
)

2023-04-06 13:43:51,894 - qm - INFO - Flags: 
2023-04-06 13:43:51,896 - qm - INFO - Sending program to QOP
2023-04-06 13:44:35,069 - qm - INFO - Executing program
Starting experimental run with id: 541. Using 'qcodes.dataset.do2d'


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

(results #541@C:\Users\ZumAdmin\Documents\GitHub\SPIN_qcodes_stations\phys-dots-40\Rafael\T7Dc6\T7D_c6_3gate_Finfet.db
 ---------------------------------------------------------------------------------------------------------------------
 LNHR_dac_chan6_V_P1 - numeric
 LNHR_dac_chan7_V_P2 - numeric
 ISD - numeric
 I - numeric,
 (None,),
 (None,))